# Machine Learning

In [2]:
import pandas as pd
# Arreglo de los dataframes para el sistema de recomendacion 
df_reviews = pd.read_parquet("reviews_eda.parquet")
df_games = pd.read_parquet("games_eda.parquet")
df_games=df_games[["item_id","title"]]
df_reviews = df_reviews[["user_id","item_id","recommend"]]
df = pd.merge(df_reviews,df_games, on="item_id", how="inner")
df = df[["user_id","title","recommend"]]
df_filtered = df[df['recommend'] == True]
df = df.astype(str)
df.to_parquet("df_ML.parquet", index= False)

In [3]:
df

,user_id,title,recommend
0,76561197970982479,Killing Floor,True
1,death-hunter,Killing Floor,True
2,DJKamBer,Killing Floor,True
3,diego9031,Killing Floor,True
4,76561198081962345,Killing Floor,True
...,...,...,...
40554,laislabonita75,Aero's Quest,True
40555,evilindiegaming,Aero's Quest,True
40556,laislabonita75,Another Perspective,False
40557,Gamer0009,The Howler,True


In [4]:
df_ml = df

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_usuario(user_id :str):
    # Construye una matriz de usuarios y juegos
    user_game_matrix = pd.crosstab(df_ml['user_id'], df_ml['title'])

    try:
        # Encuentra el índice del usuario en la matriz
        user_index = user_game_matrix.index.get_loc(user_id)
    except KeyError:
        print(f"El usuario {user_id} no está presente en los datos.")
        return None

    # Calcula la similitud de coseno entre los usuarios
    cosine_similarities = cosine_similarity(user_game_matrix, user_game_matrix)

    # Obtén las similitudes de coseno para el usuario dado
    similar_users = cosine_similarities[user_index]

    # Encuentra los juegos que el usuario no ha jugado
    games_played = user_game_matrix.loc[user_id]
    unrated_games = games_played[games_played == 0].index

    # Calcula las puntuaciones de recomendación sumando las similitudes de usuarios para los juegos no jugados
    recommendation_scores = user_game_matrix.loc[:, unrated_games].multiply(similar_users, axis=0).sum(axis=0)

    # Ordena las recomendaciones por puntuación descendente
    recommendations = recommendation_scores.sort_values(ascending=False).index.tolist()

    # Limita las recomendaciones a los primeros 5 juegos
    top_recommendations = recommendations[:5]

    return top_recommendations

# Ejemplo de uso:
# Supongamos que df es tu DataFrame con las columnas 'user_id' y 'title'
# y que contiene información sobre las interacciones usuario-juego.

# usuario_a_recomendar es el ID del usuario para el que quieres obtener recomendaciones
usuario_a_recomendar = "76561197970982479"

# Llama a la función de recomendación
recomendaciones = recomendacion_usuario(usuario_a_recomendar)

# Muestra las recomendaciones
print(f"Recomendaciones para el Usuario {usuario_a_recomendar}:")
print(recomendaciones)


Recomendaciones para el Usuario 76561197970982479:
['Team Fortress 2', "Garry's Mod", 'Counter-Strike: Global Offensive', 'Left 4 Dead 2', 'Terraria']
